<a href="https://colab.research.google.com/github/officialcyber88/Beat-Identifier-GUI/blob/main/notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# @title Mount Google Colab
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# @title Install requirements
!apt-get update && apt-get install -y libsndfile1
!pip install soundfile

!pip install --upgrade \
    gradio==3.44.0 \
    librosa==0.10.0 \
    numpy==1.24.3 \
    requests==2.31.0 \
    gdown==4.7.1 \
    torch==2.1.0

In [ ]:
# @title Beat Identifier GUI

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

import os
import tempfile
import socket
import gradio as gr
import librosa
import numpy as np
import torch
from concurrent.futures import ThreadPoolExecutor
import scipy.signal
from scipy.signal.windows import hann as _hann

# Patch SciPy so librosa.beat_track can find hann()
scipy.signal.hann = _hann

# Detect device
use_cuda = torch.cuda.is_available()
device = torch.device('cuda' if use_cuda else 'cpu')
print(f"Using device: {device}")

notes = ['C','C#','D','D#','E','F','F#','G','G#','A','A#','B']

def analyze_single(path):
    fn = os.path.basename(path)
    try:
        y, sr = librosa.load(path, sr=None)
        if y.size == 0:
            return f"Error: {fn} is empty.", None
        if use_cuda:
            import torch as _t
            y = _t.from_numpy(y).to(device).cpu().numpy()

        dur = librosa.get_duration(y=y, sr=sr)
        if dur == 0:
            return f"Error: {fn} duration is zero.", None
        m, s = divmod(int(dur), 60)

        onset_env = librosa.onset.onset_strength(y=y, sr=sr)
        bpm = 0 if onset_env.sum() == 0 else int(round(librosa.beat.beat_track(onset_envelope=onset_env, sr=sr)[0]))

        chroma = librosa.feature.chroma_cqt(y=y, sr=sr)
        key = notes[int(np.mean(chroma, axis=1).argmax())]

        tuning = librosa.estimate_tuning(y=y, sr=sr)

        summary = (
            f"**{fn}**\n\n"
            f"- Duration: {m}m {s}s ({dur:.2f}s)\n"
            f"- BPM: {bpm if bpm > 0 else '—'}\n"
            f"- Key: {key}\n"
            f"- Tuning offset: {tuning:.3f} semitones"
        )
        return summary, path

    except Exception as e:
        return f"Error analyzing {fn}: {e}", None

def analyze_batch(paths):
    with ThreadPoolExecutor(max_workers=min(4, os.cpu_count())) as ex:
        results = list(ex.map(analyze_single, paths))
    summaries, out_paths, opts = [], [], []
    for summary, p in results:
        summaries.append(summary)
        if p:
            out_paths.append(p)
            opts.append((os.path.basename(p), p))
    return "\n\n---\n\n".join(summaries), out_paths, opts

def find_available_port(start=7860, end=7900):
    for port in range(start, end + 1):
        with socket.socket() as s:
            try:
                s.bind(("", port))
                return port
            except OSError:
                continue
    return start

def process_upload(files):
    if not files:
        return "⚠ Please upload one or more MP3/WAV files.", [], []
    paths = [f.name for f in files]
    summary, out_paths, opts = analyze_batch(paths)
    default = out_paths[0] if out_paths else None
    return summary, out_paths, gr.update(choices=opts, value=default)

# === Build GUI ===
with gr.Blocks(title="Beat Identifier", theme=gr.themes.Soft()) as demo:
    gr.Markdown("# Beat Identifier")
    gr.Markdown("Drag & drop your MP3/WAV files below")
    with gr.Row():
        with gr.Column(scale=2):
            upload = gr.File(
                label="Upload Audio Files",
                file_count="multiple",
                file_types=[".mp3", ".wav"],
                type="file"
            )
            btn = gr.Button("Analyze Audio", variant="primary")
        with gr.Column(scale=3):
            gr.Markdown("### Results")
    with gr.Tabs():
        with gr.TabItem("Summary"):
            out_md = gr.Markdown("", label="Analysis Summary")
        with gr.TabItem("Files"):
            out_files = gr.File(file_count="multiple", label="Download Files")
        with gr.TabItem("Playback"):
            selector = gr.Dropdown(choices=[], label="Select File to Play")
            audio_player = gr.Audio(label="Preview", interactive=True)

    selector.change(lambda p: p or None, selector, audio_player)
    btn.click(process_upload, inputs=[upload], outputs=[out_md, out_files, selector])

    demo.queue()  # server-side queue to avoid mobile disconnects

# === Launch ===
if __name__ == "__main__":
    port = find_available_port()
    in_colab = False
    try:
        import google.colab  # type: ignore
        in_colab = True
    except ImportError:
        pass

    demo.launch(
        server_name="0.0.0.0",
        server_port=port,
        share=in_colab,   # must share=True in Colab when using queue()
        debug=in_colab    # show errors inline in Colab
    )